In [1]:
import pandas as pd
import numpy as np
from json import loads, dumps

In [2]:
table = pd.read_csv("/Users/farhadghassemi/Library/CloudStorage/OneDrive-Personal/Courses/Stanford/Conversational Virtual Assistant with Deep Learning/Project/Results/result_wikipedia_model_wikidata_compmix_test_set_gpt_domain_not_passed_generic_examples_from_WWQ_compmix_max_entities_4_extended_prompt.csv")
table.sort_values(by=['question_id'], inplace=True)

In [3]:
table.head()

,question_id,domain,question,gold_entity_id,gold_entity_label,predicted_entity_id,predicted_entity_label,predicted_entity_score
3056,6646,books,How many Pulitzer Prizes has John Updike won?,Q105756,John Updike,Q105756,John Updike,0.9955
3057,6646,books,How many Pulitzer Prizes has John Updike won?,NaN,NaN,Q46525,Pulitzer Prize,0.9161
1971,6647,books,What was the first book of John Updike to win ...,Q105756,John Updike,Q105756,John Updike,0.9955
1972,6647,books,What was the first book of John Updike to win ...,NaN,NaN,Q46525,Pulitzer Prize,0.9161
1937,6648,books,Which other book than Rabbit Is Rich has won a...,NaN,NaN,Q46525,Pulitzer Prize,0.9161


In [4]:
table['predicted_entities'] = table[['predicted_entity_label', 'predicted_entity_id']].values.tolist()
table['gold_entities'] = table[['gold_entity_label', 'gold_entity_id']].values.tolist()
table.drop(['gold_entity_id', 'gold_entity_label', 'predicted_entity_id', 'predicted_entity_label', 'predicted_entity_score'], axis=1, inplace=True)

In [5]:
table.head()

,question_id,domain,question,predicted_entities,gold_entities
3056,6646,books,How many Pulitzer Prizes has John Updike won?,"[John Updike, Q105756]","[John Updike, Q105756]"
3057,6646,books,How many Pulitzer Prizes has John Updike won?,"[Pulitzer Prize, Q46525]","[nan, nan]"
1971,6647,books,What was the first book of John Updike to win ...,"[John Updike, Q105756]","[John Updike, Q105756]"
1972,6647,books,What was the first book of John Updike to win ...,"[Pulitzer Prize, Q46525]","[nan, nan]"
1937,6648,books,Which other book than Rabbit Is Rich has won a...,"[Pulitzer Prize, Q46525]","[nan, nan]"


In [6]:
table2 = table.groupby("question_id").agg({'domain': 'first', 'question': 'first', 'predicted_entities': lambda x: list(x), 'gold_entities': lambda x: list(x)})
table2['gold_entities'] = table2['gold_entities'].apply(lambda x: [xi for xi in x if not pd.isna(xi[0])])
table2['predicted_entities'] = table2['predicted_entities'].apply(lambda x: [xi for xi in x if not pd.isna(xi[0])])
table2['question_id'] = table2.index

In [7]:
table2.head()

,domain,question,predicted_entities,gold_entities,question_id
question_id,,,,,
6646,books,How many Pulitzer Prizes has John Updike won?,"[[John Updike, Q105756], [Pulitzer Prize, Q465...","[[John Updike, Q105756]]",6646
6647,books,What was the first book of John Updike to win ...,"[[John Updike, Q105756], [Pulitzer Prize, Q465...","[[John Updike, Q105756]]",6647
6648,books,Which other book than Rabbit Is Rich has won a...,"[[Pulitzer Prize, Q46525], [Rabbit Is Rich, Q5...","[[John Updike, Q105756], [Rabbit Is Rich, Q549...",6648
6649,books,In which year was the book Rabbit at Rest publ...,"[[Rabbit at Rest, Q1649823]]","[[Rabbit at Rest, Q1649823]]",6649
6650,books,When did John Updike pass away?,"[[John Updike, Q105756]]","[[John Updike, Q105756]]",6650


In [8]:
result = table2.to_json("/Users/farhadghassemi/Library/CloudStorage/OneDrive-Personal/Courses/Stanford/Conversational Virtual Assistant with Deep Learning/Project/Results/result_wikipedia_model_wikidata_compmix_test_set_gpt_domain_not_passed_generic_examples_from_WWQ_compmix_max_entities_4_extended_prompt.json", orient="records", indent=3)